In [18]:
import pandas as pd
import datetime

In [21]:
df_old = pd.read_csv('cleaned_data.csv')
print(df_old.shape)
print(df_old.head())

(5449, 4)
  executive_order_number                                              title  \
0                   7532  Establishing Shinnecock Migratory Bird Refuge,...   
1                   7533  Exemption of James E. Tibbitts From Compulsory...   
2                   7534  Excluding Land From Sitgreaves National Forest...   
3                   7535  Partial Revocation of   6110 of May 2, 1933, W...   
4                   7536  Partial Revocation of   5687 of August 18, 193...   

       signing_date                                  disposition_notes  
0   January 8, 1937  Federal Register page and date: 2 FR 63, Janua...  
1  January 11, 1937  Note: This Executive order was not received fo...  
2  January 12, 1937  Federal Register page and date: 2 FR 87, Janua...  
3  January 12, 1937  Federal Register page and date: 2 FR 88, Janua...  
4  January 12, 1937  Federal Register page and date: 2 FR 88, Janua...  


In [22]:
df_old['signing_date'] = pd.to_datetime(df_old['signing_date'])

In [23]:
print(df_old.head())

  executive_order_number                                              title  \
0                   7532  Establishing Shinnecock Migratory Bird Refuge,...   
1                   7533  Exemption of James E. Tibbitts From Compulsory...   
2                   7534  Excluding Land From Sitgreaves National Forest...   
3                   7535  Partial Revocation of   6110 of May 2, 1933, W...   
4                   7536  Partial Revocation of   5687 of August 18, 193...   

  signing_date                                  disposition_notes  
0   1937-01-08  Federal Register page and date: 2 FR 63, Janua...  
1   1937-01-11  Note: This Executive order was not received fo...  
2   1937-01-12  Federal Register page and date: 2 FR 87, Janua...  
3   1937-01-12  Federal Register page and date: 2 FR 88, Janua...  
4   1937-01-12  Federal Register page and date: 2 FR 88, Janua...  


In [25]:
print(df_old.sort_values(by='signing_date', ascending = True).head())

  executive_order_number                                              title  \
0                   7532  Establishing Shinnecock Migratory Bird Refuge,...   
1                   7533  Exemption of James E. Tibbitts From Compulsory...   
2                   7534  Excluding Land From Sitgreaves National Forest...   
3                   7535  Partial Revocation of   6110 of May 2, 1933, W...   
4                   7536  Partial Revocation of   5687 of August 18, 193...   

  signing_date                                  disposition_notes  
0   1937-01-08  Federal Register page and date: 2 FR 63, Janua...  
1   1937-01-11  Note: This Executive order was not received fo...  
2   1937-01-12  Federal Register page and date: 2 FR 87, Janua...  
3   1937-01-12  Federal Register page and date: 2 FR 88, Janua...  
4   1937-01-12  Federal Register page and date: 2 FR 88, Janua...  


In [ ]:
#import data from bulk CSV downloads

file_prefix = 'documents_signed_by_'
pres_names = ['joseph_r_biden_jr', 'donald_trump', 'barack_obama', 'george_w_bush', 'william_j_clinton']
file_suffix = '_of_type_presidential_document_and_of_presidential_document_type_executive_order.csv'
colnames = ['executive_order_number', 'title', 'signing_date', 'disposition_notes']

df_all = pd.DataFrame(columns = colnames)
for name in pres_names:
    filename = file_prefix + name + file_suffix
    df = pd.read_csv(filename)
    df_subset = df[colnames]
    print(name)
    df_all = df_all.append(df_subset)
print(df_all.head())
print(df_all.shape)